In [ ]:
import hmftpy as hmf
import numpy as np
import matplotlib.pyplot as plt
from hmftpy.plaquettes.square import plaq4, plaq9
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm

In [ ]:
b4 = spin_basis_1d(4, pauli=1)
b9 = spin_basis_1d(9, pauli=1)
b16 = spin_basis_1d(16, pauli=1)

# Hamiltonian
Here, we consider the Hamiltonian
\begin{align*}
    H = &\sum_{\langle i, j\rangle} \vec D_{ij} \cdot \left(\vec \sigma_i \times\vec \sigma_j\right)
        + \sum_{\langle i, j\rangle} J_{ij} \vec \sigma_i \cdot \vec \sigma_j
        + B \sum_i \sigma_i^z.
\end{align*}
where $\vec D_{ij}=D\hat z$ for nearest-neighbor bonds $\langle i,j \rangle$ and zero elsewise, and similarly $J_{ij}=J$ for $\langle i, j\rangle$ nearest-neighbors. For now, I will set $B=0$.


In [ ]:
J = -0.2
D = 0.1

interactions = {'x_bonds': {'xx': J, 'yy': J, 'zz': J,
                            'xy': D, 'yx': -D},
                'y_bonds': {'xx': J, 'yy': J, 'zz': J,
                            'xy': D, 'yx': -D},
               }

In [ ]:
Hi = hmf.operators.inner_hamiltonian(plaq4, interactions, b4, checks=False)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq4, interactions, b4, checks=False)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq4, interactions, b4)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

Let's look at observables as we change $D$. 

Specifically, let's measure scalar chirality
$\chi = \sigma_0 \cdot \left(\sigma_1 \times \sigma_2\right)$ and magnetization $M_z = \sum_i \sigma_i^z$.

In [ ]:
Mz = quantum_operator({'static': [['z', [[1./4, i] for i in range(4)]]]}, basis=b4, check_herm=False, check_symm=False)
tris = [[0,1,2], [1,0,3], [2,3,0], [3,2,1]]
# tris = [[0,1,3], [1,2,4], [2,0,5],
#         [3,4,6], [4,5,7], [5,3,8],
#         [6,7,0], [7,8,1], [8,6,2]]
chi = quantum_operator({'static': [['xyz', [[1,t[0],t[1],t[2]] for t in tris]],
                                   ['xzy', [[-1,t[0],t[1],t[2]] for t in tris]],
                                   ['yzx', [[1,t[0],t[1],t[2]] for t in tris]],
                                   ['yxz', [[-1,t[0],t[1],t[2]] for t in tris]],
                                   ['zxy', [[1,t[0],t[1],t[2]] for t in tris]],
                                   ['zyx', [[-1,t[0],t[1],t[2]] for t in tris]]]}, basis=b4,
                      check_herm=False, check_symm=False)
steps = 20

thetas = np.linspace(0, np.pi, steps)
J0 = 1
D0 = 1
B = 0.001
mf_f = None
mf_r = None

vs_f = np.zeros((steps, b4.Ns), dtype=np.complex128)
vs_r = np.zeros((steps, b4.Ns), dtype=np.complex128)
vs_p = np.zeros((steps, b4.Ns), dtype=np.complex128)
es_f = np.zeros(steps)
es_r = np.zeros(steps)
es_p = np.zeros(steps)
for i, theta in enumerate(thetas):
    D = D0*np.sin(theta)
    J = -J0*np.cos(theta)
    Dr = D0*np.sin(thetas[steps-1-i])
    Jr = J0*np.cos(thetas[steps-1-i])
    
    interactions = {'x_bonds': {'xx': J, 'yy': J, 'zz': J,
                                'xy': D, 'yx': -D},
                    'y_bonds': {'xx': J, 'yy': J, 'zz': J,
                                'xy': D, 'yx': -D},
                    'local': {'z': -B}
                    }    
    Hp = hmf.operators.periodic_hamiltonian(plaq4, interactions, b4, checks=False)
    es, vs = Hp.eigh()
    vs_p[i,:] = vs[:,0]
    es_p[i] = es[0]
    es_f[i], vs_f[i,:], mf_f, cvg = hmf.do_hmft(plaq4, interactions, b4, mf0=mf_f)
    interactions = {'x_bonds': {'xx': Jr, 'yy': Jr, 'zz': Jr,
                                'xy': Dr, 'yx': -Dr},
                    'y_bonds': {'xx': Jr, 'yy': Jr, 'zz': Jr,
                                'xy': Dr, 'yx': -Dr},
                    'local': {'z': -B}
                    }   
    es_r[i], vs_r[i,:], mf_r, cvg = hmf.do_hmft(plaq4, interactions, b4, mf0=mf_r)

In [ ]:
chis_hmft = np.zeros(steps)
chis_pbc = np.zeros(steps)
ms_hmft = np.zeros(steps)
ms_pbc = np.zeros(steps)
inds = np.zeros(steps, dtype=int)
for i in range(steps):
    if es_f[i]-es_r[::-1][i] < 10**-7:
        inds[i] = 1
# inds = np.argmin([es_f, es_r[::-1]], axis=0)
print(inds)
for i, ind in enumerate(inds):
    vp = vs_p[i,:]
    vh = [vs_f[i,:], vs_r[steps-1-i,:]][ind]
    ms_hmft[i] = Mz.expt_value(vh).real
    ms_pbc[i] = Mz.expt_value(vp).real
    chis_hmft[i] = chi.expt_value(vh).real
    chis_pbc[i] = chi.expt_value(vp).real

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(thetas, es_p, marker='+', label='PBC')
plt.scatter(thetas, es_f, marker='x', label='HMFT1')
plt.scatter(thetas, es_r[::-1], marker='.', label='HMFT2')
plt.plot(thetas, np.min([es_f, es_r[::-1]], axis=0))
plt.legend()
plt.xlabel(r'$\theta$')
plt.ylabel(r'$e$')
plt.show()

plt.figure(figsize=(4,3), dpi=200)
plt.plot(thetas, chis_pbc, marker='.', label='PBC')
plt.plot(thetas, chis_hmft, marker='+', label='HMFT')
plt.xlabel(r'$\theta$')
plt.ylabel(r'$\chi$')
plt.legend()
plt.show()

plt.figure(figsize=(4,3), dpi=200)
plt.plot(thetas, ms_pbc, marker='.', label='PBC')
plt.plot(thetas, ms_hmft, marker='+', label='HMFT')
plt.xlabel(r'$\theta$')
plt.ylabel(r'$M_z$')
plt.legend()
plt.show()